In [29]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder \
  .appName('clean_products') \
  .config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar') \
  .getOrCreate()  
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [30]:
table = "becade_rgarciaf.stg_compras"
stg_compras = spark.read \
  .format("bigquery") \
  .option("table", table) \
  .load()


In [3]:
table = "becade_rgarciaf.compras_anuales"
compras_anuales_bq = spark.read \
  .format("bigquery") \
  .option("table", table) \
  .load()


In [31]:
from pyspark.sql.functions import current_date, date_sub,add_months,col
from pyspark.sql import functions as F
import pandas as pd
from pyspark.sql.functions import year, month, dayofmonth,col
from pyspark.sql.functions import sum, col, desc, avg, count

In [32]:
stg_compras_2 = stg_compras.select("compra_id","client_id","product_id","cantidad","precio","isprime","metodo_pago",
    year("fecha_compra").alias('year'), 
    month("fecha_compra").alias('month'), 
    dayofmonth("fecha_compra").alias('day')
)

In [33]:
stg_compras_2.orderBy("compra_id").show(10)

+-------------------+-----------------+----------+--------+------+-------+-----------+----+-----+---+
|          compra_id|        client_id|product_id|cantidad|precio|isprime|metodo_pago|year|month|day|
+-------------------+-----------------+----------+--------+------+-------+-----------+----+-----+---+
|209-0000191-5317909|209-931475-61-720|B08L7F3CDV|       1|  9.99|   true|  **** 2800|2019|    1| 18|
|209-0000670-1949568|209-931475-61-720|B08L7F3CDV|       1|  9.99|   true|  **** 2800|2021|   11| 28|
|209-0000865-5872439|209-931475-61-720|B097Y38X79|       1| 46.59|   true|  **** 2800|2018|    4| 15|
|209-0000865-5872439|209-931475-61-720|B09BNK4592|       1| 59.88|   true|  **** 2800|2018|    4| 15|
|209-0001380-0732771|209-696678-32-117|B08FBWB96P|       1| 29.99|   true|  **** 7589|2016|    9| 15|
|209-0001882-2966971|209-931475-61-720|B072QN7FWK|       1| 49.99|   true|  **** 2800|2013|    7| 11|
|209-0001952-8693818|209-931475-61-720|B07VJRZ62R|       1|299.99|   true|  **** 2

In [34]:
data=[["1"]]
current_year=spark.createDataFrame(data,["id"])
current_year.withColumn("current_date",date_sub(current_date(),1))

id,current_date
1,2022-03-01


In [35]:
current_year_2 = current_year.select(
    year("current_date").alias('year'), 
    month("current_date").alias('month'), 
    dayofmonth("current_date").alias('day')
)

In [36]:
current_year_number = current_year_2.select("year").collect()

In [37]:
count_x = current_year_number[0].year

In [38]:
stg_compras_3 = stg_compras_2.filter(stg_compras_2["year"]==count_x)

In [39]:
stg_compras_3.printSchema()

root
 |-- compra_id: string (nullable = true)
 |-- client_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- cantidad: long (nullable = true)
 |-- precio: double (nullable = true)
 |-- isprime: boolean (nullable = true)
 |-- metodo_pago: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)



In [40]:
stg_compras_4=stg_compras_3.withColumn("cantidad",stg_compras_3.cantidad.cast("int"))

In [14]:
################################################## compras_anuales ##################################################

In [15]:
compras_3 = stg_compras_4.withColumn("cantidad",stg_compras_4.cantidad.cast("int"))\
    .withColumn("precio",stg_compras_4.precio.cast("float"))

In [16]:
compras_anuales = compras_3.groupBy("year")\
    .agg(sum("cantidad").alias("venta_total"),count("compra_id").alias("total_compras")) 

In [17]:
compras_promedio_ventas_mensual_anual = compras_3.groupBy("month","year")\
    .agg(sum("cantidad").alias("avg_venta_mensual")) 

In [18]:
compras_promedio_ventas_mensual_anual_2 = compras_promedio_ventas_mensual_anual.groupBy("year")\
    .agg(avg("avg_venta_mensual").alias("avg_venta_mensual")) 

In [19]:
 compras_anuales_2 = compras_anuales.join(compras_promedio_ventas_mensual_anual_2,compras_anuales.year ==  compras_anuales.year,"outer").select(compras_anuales.year, compras_anuales.venta_total,compras_anuales.total_compras,compras_promedio_ventas_mensual_anual_2.avg_venta_mensual)

In [20]:
compras_anuales_2=compras_anuales_2.withColumn("avg_venta_mensual",compras_anuales_2.avg_venta_mensual.cast('int')) \

In [21]:
compras_anuales_2.show()

+----+-----------+-------------+-----------------+
|year|venta_total|total_compras|avg_venta_mensual|
+----+-----------+-------------+-----------------+
|2022|        379|          351|              189|
+----+-----------+-------------+-----------------+



In [22]:
compras_anuales_bq.show(20)

+----+-----------+-------------+-----------------+
|year|venta_total|total_compras|avg_venta_mensual|
+----+-----------+-------------+-----------------+
|2011|     190838|       178561|            15903|
|2019|     190901|       178539|            15908|
|2018|     189889|       177502|            15824|
|2020|     193183|       180368|            16098|
|2014|     189880|       177407|            15823|
|2017|     192075|       179649|            16006|
|2010|     189442|       177276|            15786|
|2013|     191325|       178695|            15943|
|2016|     191064|       178483|            15922|
|2015|     191521|       178865|            15960|
|2012|     193062|       180462|            16088|
|2022|        169|          159|              169|
+----+-----------+-------------+-----------------+



In [23]:
compras_anuales_bq_2 = compras_anuales_bq.filter(compras_anuales_bq.year<count_x).orderBy("year")

In [24]:
compras_anuales_3 = compras_anuales_bq_2.union(compras_anuales_2)

In [25]:
compras_anuales_3.show(20)

+----+-----------+-------------+-----------------+
|year|venta_total|total_compras|avg_venta_mensual|
+----+-----------+-------------+-----------------+
|2010|     189442|       177276|            15786|
|2011|     190838|       178561|            15903|
|2012|     193062|       180462|            16088|
|2013|     191325|       178695|            15943|
|2014|     189880|       177407|            15823|
|2015|     191521|       178865|            15960|
|2016|     191064|       178483|            15922|
|2017|     192075|       179649|            16006|
|2018|     189889|       177502|            15824|
|2019|     190901|       178539|            15908|
|2020|     193183|       180368|            16098|
|2022|        379|          351|              189|
+----+-----------+-------------+-----------------+



In [26]:
compras_anuales_3.write \
  .format("bigquery") \
  .option("table","becade_rgarciaf.compras_anuales") \
  .option("temporaryGcsBucket", "amazon_bucket_ramiro") \
  .mode('overwrite') \
  .save()

In [ ]:
################################################## compras_mensuales ##################################################

In [41]:
stg_compras_2.show(10)

+-------------------+-----------------+----------+--------+------+-------+-----------+----+-----+---+
|          compra_id|        client_id|product_id|cantidad|precio|isprime|metodo_pago|year|month|day|
+-------------------+-----------------+----------+--------+------+-------+-----------+----+-----+---+
|623-4247690-9632191|623-196004-19-984|B00N69D6AS|       1|236.99|  false|  **** 0185|2021|    1|  1|
|623-7594898-2072579|623-196004-19-984|B00N69D6AS|       1|236.99|  false|  **** 0185|2021|    1|  6|
|623-7531422-9031402|623-196004-19-984|B00N69D6AS|       1|236.99|  false|  **** 0185|2021|    1| 21|
|623-1709124-5053980|623-196004-19-984|B00N69D6AS|       1|236.99|  false|  **** 0185|2021|    2|  3|
|623-4609449-6977067|623-196004-19-984|B00N69D6AS|       1|236.99|  false|  **** 0185|2021|    2|  6|
|623-0020193-2513734|623-196004-19-984|B00N69D6AS|       1|236.99|  false|  **** 0185|2021|    2| 17|
|623-9262920-5949429|623-196004-19-984|B00N69D6AS|       1|236.99|  false|  **** 0

In [42]:
stg_compras_mensuales = stg_compras_2.filter(stg_compras_2["year"].between(count_x-1,count_x))

In [43]:
stg_compras_mensuales.orderBy("client_id").show(10)

+-------------------+-----------------+----------+--------+------+-------+-----------+----+-----+---+
|          compra_id|        client_id|product_id|cantidad|precio|isprime|metodo_pago|year|month|day|
+-------------------+-----------------+----------+--------+------+-------+-----------+----+-----+---+
|209-7284389-8989855|209-696678-32-117|B00N69D6AS|       1|236.99|   true|  **** 7589|2021|    1| 24|
|209-8246591-8104443|209-696678-32-117|B00N69D6AS|       1|236.99|   true|  **** 7589|2021|    6| 20|
|209-6485384-5644016|209-696678-32-117|B00N69D6AS|       1|236.99|   true|  **** 7589|2021|    1| 30|
|209-6495389-1336848|209-696678-32-117|B00N69D6AS|       1|236.99|   true|  **** 7589|2021|    2| 26|
|209-3265508-3692436|209-696678-32-117|B00N69D6AS|       1|236.99|   true|  **** 7589|2021|    3| 14|
|209-3828490-4192462|209-696678-32-117|B00N69D6AS|       1|236.99|   true|  **** 7589|2021|    4|  3|
|209-8899707-4226127|209-696678-32-117|B00N69D6AS|       1|236.99|   true|  **** 7

In [44]:
compras_mensuales =  stg_compras_mensuales.groupBy("month","year")\
    .agg(sum("cantidad").alias("venta_total_mes"),count("compra_id").alias("total_compras_mes")) 

In [45]:
import pyspark.sql.functions as func
from pyspark.sql.window import Window
from pyspark.sql.functions import lit

In [60]:
compras_mensuales_2 = compras_mensuales.withColumn('venta_total_mes_anterior',func.lag(compras_mensuales['venta_total_mes']).over(Window.orderBy("month","year")))

In [53]:
current_year_number = current_year_2.select("month").collect()
count_month = current_year_number[0].month

In [57]:
compras_mensuales_3 = compras_mensuales_2.filter( (stg_compras_2["year"]==count_x) & (stg_compras_2["month"]==count_month))

In [61]:
compras_mensuales_2.show(10)

+-----+----+---------------+-----------------+------------------------+
|month|year|venta_total_mes|total_compras_mes|venta_total_mes_anterior|
+-----+----+---------------+-----------------+------------------------+
|    1|2021|          12984|            12175|                    null|
|    1|2022|          12898|            12050|                   12984|
|    2|2021|          13190|            12313|                   12898|
|    2|2022|          12605|            11800|                   13190|
|    3|2021|          11739|            10928|                   12605|
|    3|2022|            407|              372|                   11739|
|    4|2021|          17642|            16442|                     407|
|    5|2021|          15957|            14904|                   17642|
|    6|2021|          20499|            19109|                   15957|
|    7|2021|          19931|            18606|                   20499|
+-----+----+---------------+-----------------+------------------

In [59]:
compras_mensuales_3.show(12)

+-----+----+---------------+-----------------+------------------------+
|month|year|venta_total_mes|total_compras_mes|venta_total_mes_anterior|
+-----+----+---------------+-----------------+------------------------+
|    3|2022|            407|              372|                   11739|
+-----+----+---------------+-----------------+------------------------+



In [62]:
compras_mensuales_3.write \
  .format("bigquery") \
  .option("table","becade_rgarciaf.compras_mensuales") \
  .option("temporaryGcsBucket", "amazon_bucket_ramiro") \
  .mode('append') \
  .save()